In [ ]:
# Network Analysis Notebook
import networkx as nx
import pandas as pd
from src.utils import compute_correlations, build_network
from src.visualization import plot_network

data = pd.read_csv("../data/processed/nv_center_data_processed.csv")

correlations = compute_correlations(data)
G = build_network(correlations)

plot_network(G)